In [1]:
# Gọi trực tiếp API Nhanh.vn để xem response khi nhập Bill ID
import requests
import json
from pprint import pprint

# Cấu hình API Nhanh.vn (lấy từ config)
API_URL = "https://open.nhanh.vn/api/bill/imexrequirements"
API_CONFIG = {
    "version": "2.0",
    "appId": "74951",
    "businessId": "8901",
    "accessToken": "twf9P1xFZCUUgwt8zR0XgNeB6V5jsbq2KHb14bxovqK1ppCxyADwOK8FzQlCEeEGABRZINXoUCSzM50kjhwcrUSBWTY5nSvyhfnH2X2cI0pC7pNczSVxc1ratdDmxF85q7hUTUNCrUnpPTG5ZwLNO7bkMlEEJTCdPhgYaC"
}

print("=" * 70)
print("TEST API NHANH.VN: /api/bill/imexrequirements")
print("=" * 70)
print(f"API URL: {API_URL}\n")


TEST API NHANH.VN: /api/bill/imexrequirements
API URL: https://open.nhanh.vn/api/bill/imexrequirements



In [2]:
def call_nhanh_api(bill_id):
    """Gọi API Nhanh.vn và hiển thị response đầy đủ"""
    print(f"\n{'='*70}")
    print(f"BILL ID: {bill_id}")
    print(f"{'='*70}\n")
    
    try:
        # Chuẩn bị payload
        payload = {
            **API_CONFIG,
            "data": json.dumps({"billId": int(bill_id)})
        }
        
        print("📤 Request Payload:")
        print(f"   URL: {API_URL}")
        print(f"   Method: POST")
        print(f"   Data: {json.dumps(payload, indent=2, ensure_ascii=False)}")
        print()
        
        # Gọi API
        response = requests.post(API_URL, data=payload)
        
        print(f"📥 Response Status Code: {response.status_code}")
        print()
        
        # Hiển thị raw response
        print("=" * 70)
        print("RAW RESPONSE (JSON):")
        print("=" * 70)
        try:
            response_json = response.json()
            print(json.dumps(response_json, indent=2, ensure_ascii=False))
        except json.JSONDecodeError:
            print("⚠️ Response không phải JSON:")
            print(response.text[:1000])
            return None
        
        print()
        print("=" * 70)
        print("PARSED DATA:")
        print("=" * 70)
        
        # Parse và hiển thị dữ liệu
        if response_json.get("code") == 1:
            data = response_json.get("data")
            if not data:
                print("❌ Không có dữ liệu (data is null)")
                return None
            
            imexs = data.get("imexs")
            if not imexs:
                print("❌ Không có imexs (imexs is null)")
                return None
            
            print(f"✅ Tìm thấy {len(imexs)} dòng sản phẩm\n")
            
            # Hiển thị từng item
            result = []
            for idx, (key, item) in enumerate(imexs.items(), 1):
                print(f"\n--- Item {idx} (key: {key}) ---")
                parsed_item = {
                    "requiredQuantity": item.get("requiredQuantity", ""),
                    "damagedQuantity": item.get("damagedQuantity", ""),
                    "approvedQuantity": item.get("approvedQuantity", ""),
                    "realQuantity": item.get("realQuantity", ""),
                    "approvedByUser": item.get("approvedByUser", ""),
                    "requiredAt": item.get("requiredAt", ""),
                    "approvedAt": item.get("approvedAt", ""),
                    "confirmedAt": item.get("confirmedAt", ""),
                    "fromDepotId": item.get("fromDepotId", ""),
                    "fromDepotName": item.get("fromDepotName", ""),
                    "toDepotId": item.get("toDepotId", ""),
                    "toDepotName": item.get("toDepotName", ""),
                    "status": item.get("status", "")
                }
                
                # Highlight thông tin kho đến
                print(f"  🏭 Kho đi: {parsed_item['fromDepotName']} (ID: {parsed_item['fromDepotId']})")
                print(f"  🏭 Kho đến: {parsed_item['toDepotName']} (ID: {parsed_item['toDepotId']}) ⭐")
                print(f"  📊 Status: {parsed_item['status']}")
                print(f"  📦 Số lượng yêu cầu: {parsed_item['requiredQuantity']}")
                print(f"  📦 Số lượng hỏng: {parsed_item['damagedQuantity']}")
                print(f"  📦 Số lượng duyệt: {parsed_item['approvedQuantity']}")
                print(f"  📦 Số lượng thực tế: {parsed_item['realQuantity']}")
                
                result.append(parsed_item)
            
            # Tóm tắt
            print(f"\n{'='*70}")
            print("TÓM TẮT:")
            print(f"{'='*70}")
            if result:
                first_item = result[0]
                print(f"✅ Bill ID hợp lệ")
                print(f"   Kho đi: {first_item['fromDepotName']} ({first_item['fromDepotId']})")
                print(f"   Kho đến: {first_item['toDepotName']} ({first_item['toDepotId']}) ⭐")
                
                # Kiểm tra status
                all_statuses = {int(item.get("status", 0)) for item in result}
                valid_statuses = {3, 4, 5, 6}
                is_valid = any(status in valid_statuses for status in all_statuses)
                print(f"   Status: {all_statuses} {'✅ Hợp lệ' if is_valid else '❌ Chưa được duyệt'}")
            
            return result
        else:
            error_msg = response_json.get("messages", "Unknown error")
            print(f"❌ API trả về lỗi:")
            print(f"   Code: {response_json.get('code')}")
            print(f"   Message: {error_msg}")
            return None
            
    except ValueError as e:
        print(f"❌ Lỗi: Bill ID phải là số nguyên ({str(e)})")
        return None
    except requests.exceptions.RequestException as e:
        print(f"❌ Lỗi kết nối: {str(e)}")
        return None
    except Exception as e:
        print(f"❌ Lỗi không xác định: {str(e)}")
        import traceback
        traceback.print_exc()
        return None
    
    print()


In [6]:
# Test với Bill ID - Thay đổi số này để test
bill_id = "7793131"  # ⬅️ THAY ĐỔI SỐ NÀY ĐỂ TEST

result = call_nhanh_api(bill_id)



BILL ID: 7793131

📤 Request Payload:
   URL: https://open.nhanh.vn/api/bill/imexrequirements
   Method: POST
   Data: {
  "version": "2.0",
  "appId": "74951",
  "businessId": "8901",
  "accessToken": "twf9P1xFZCUUgwt8zR0XgNeB6V5jsbq2KHb14bxovqK1ppCxyADwOK8FzQlCEeEGABRZINXoUCSzM50kjhwcrUSBWTY5nSvyhfnH2X2cI0pC7pNczSVxc1ratdDmxF85q7hUTUNCrUnpPTG5ZwLNO7bkMlEEJTCdPhgYaC",
  "data": "{\"billId\": 7793131}"
}

📥 Response Status Code: 200

RAW RESPONSE (JSON):
{
  "code": 1,
  "data": {
    "totalPages": 1,
    "page": 1,
    "imexs": {
      "68476129": {
        "id": 68476129,
        "billId": 7793131,
        "type": 2,
        "mode": 3,
        "fromDepotId": 170507,
        "toDepotId": 142681,
        "supplierId": null,
        "productId": 37932386,
        "imeiId": null,
        "productBatchId": null,
        "discount": null,
        "requiredQuantity": 10,
        "damagedQuantity": null,
        "requiredPrice": 162945,
        "requiredDescription": "AL - 5068 ( THL ) - chả